In [1]:
import os, zipfile
import pandas as pd
import numpy as np

In [2]:
zip_path='../Data/Archive.zip'
zf = zipfile.ZipFile(zip_path)
method_files = [(f , f.split('/')[-3], f.split('/')[-2], "method.csv") for f in zf.namelist() if f.endswith('method.csv')]
class_files = [(f , f.split('/')[-3], f.split('/')[-2], "class.csv") for f in zf.namelist() if f.endswith('class.csv')]
variable_files = [(f , f.split('/')[-3], f.split('/')[-2], "variable.csv") for f in zf.namelist() if f.endswith('variable.csv')]
field_files = [(f , f.split('/')[-3], f.split('/')[-2], "field.csv") for f in zf.namelist() if f.endswith('field.csv')]

df = pd.DataFrame(method_files + class_files + variable_files + field_files, columns=["full_path", "class", "project_name", "file_type"])

In [3]:
len(class_files)

69

In [4]:
label_match_df = pd.read_csv('../Data/ck_results_of_repos.csv')
label_match_df.drop_duplicates(inplace=True)
label_match_df

,folder,pattern
0,ck_metrics_root/2048-android-master,NONE
1,ck_metrics_root/AdAway-master,MVVM
2,ck_metrics_root/amdroid-master,NONE
3,ck_metrics_root/android-adbm-master,NONE
4,ck_metrics_root/android-applications-info-master,NONE
...,...,...
64,ck_metrics_root/Todoer-master,NONE
65,ck_metrics_root/yubiclip-android-master,MVC
66,ck_metrics_root/zanavi-master,MVC
67,ck_metrics_root/Zensuren-master,MVC


In [5]:
label_match_df['folder'] = label_match_df['folder'].apply(lambda x : x.replace('ck_metrics_root/',''))
from_file2Class = dict(zip(label_match_df['folder'], label_match_df['pattern']))

In [6]:
X = []
Y = []
i = 0
cols = []
for project_name, project_samples in df.groupby("project_name"):
    project_data = []
    failed = 0
    class_name = from_file2Class[project_name]
    for path in project_samples.sort_values(by="file_type")[['full_path','file_type']].values:
        dddf=pd.read_csv(zf.open(path[0])).select_dtypes(exclude=[object]).fillna(0).mean()
        if len(dddf) == 0:
            failed+=1
            if i == 0: cols = []
            break
        project_data += dddf.to_list()
        if i == 0:
            cols += [f'{path[-1][:-4]}_{cname}' for cname in dddf.index]
    if failed == 0:
        i+=1
        project_data = np.array(project_data).reshape(1,-1)
        X += project_data.tolist()
        Y.append(class_name)
        
print(f"Total {i}")
# return np.array(X), Y, cols

Total 69


In [8]:
res_df = pd.DataFrame(X, columns=cols)
res_df['label'] = Y

In [12]:
res_df.to_csv('../Data/coach_data.csv', index=False)